<a href="https://colab.research.google.com/github/nedru004/LibraryAnalysis_gui/blob/main/LibraryAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Download LibraryAnalysis GitHub

#@markdown Please execute each cell by pressing the *Play* button on
#@markdown the left.

!git clone 'https://github.com/nedru004/LibraryAnalysis_gui'
!pip3 install -r ./LibraryAnalysis_gui/requirements.txt

In [ ]:
#@title 2. Upload WT fasta file ⬆️
from google.colab import files
use_google_drive = True  #@param {type:"boolean"}
if use_google_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  wt_name = '/content/gdrive/MyDrive/data/otto_12/tcb.fa'  #@param {type:"string"}
else:
  from google.colab import files
  import os
  dir_path = os.path.dirname(os.path.realpath('__file__'))
  uploaded = files.upload()
  wt_name = list(uploaded.keys())[0]
  file1 = open(wt_name, "wb")
  file1.write(uploaded[wt_name])
  file1.close()
  wt_name = os.path.join(dir_path,wt_name)

In [ ]:
#@title 3. Upload read1 ⬆️
from google.colab import files
use_google_drive = True  #@param {type:"boolean"}
if use_google_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  read1_name = '/content/gdrive/MyDrive/data/otto_12/42A-A-Neg_S34_L002_R1_001.fastq.gz'  #@param {type:"string"}
else:
  read1_name = list(uploaded.keys())[0]
  file1 = open(read1_name, "wb")
  file1.write(uploaded[read1_name])
  file1.close()
  read1_name = os.path.join(dir_path,read1_name)

In [ ]:
#@title 4. Upload read2 ⬆️
use_google_drive = True  #@param {type:"boolean"}
if use_google_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  read2_name = '/content/gdrive/MyDrive/data/otto_12/42A-A-Neg_S34_L002_R2_001.fastq.gz'  #@param {type:"string"}
else:
  uploaded = files.upload()
  read2_name = list(uploaded.keys())[0]
  file1 = open(read2_name, "wb")
  file1.write(uploaded[read2_name])
  file1.close()
  read2_name = os.path.join(dir_path,read2_name)

In [ ]:
#@title 4. Upload inserted domain file ⬆️
use_google_drive = False  #@param {type:"boolean"}
if use_google_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  domains = '/content/LibraryAnalysis_gui/scripts/domains.txt'  #@param {type:"string"}
else:
  uploaded = files.upload()
  tmp_name = list(uploaded.keys())[0]
  file1 = open(tmp_name, "wb")
  file1.write(uploaded[tmp_name])
  file1.close()
  domains = os.path.join(dir_path,tmp_name)

In [ ]:
#@title 4. Upload read2 ⬆️
use_google_drive = False  #@param {type:"boolean"}
if use_google_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  designed_mutations = '/content/LibraryAnalysis_gui/scripts/designed_mutations.txt'  #@param {type:"string"}
else:
  uploaded = files.upload()
  tmp_name = list(uploaded.keys())[0]
  file1 = open(tmp_name, "wb")
  file1.write(uploaded[tmp_name])
  file1.close()
  designed_mutations = os.path.join(dir_path,tmp_name)

In [ ]:
#@title 5. Run ⭐
run_variant = True  #@param {type:"boolean"}
run_correlation = False  #@param {type:"boolean"}
indel_analysis = True  #@param {type:"boolean"}
full_length_analysis = True  #@param {type:"boolean"}
long_read_analysis = False  #@param {type:"boolean"}
minimum_read_quality = 15  #@param {type:"slider", min:0, max:100, step:1}
minimum_base_quality = 10  #@param {type:"slider", min:0, max:100, step:1}
parallel_threads = 1  #@param {type:"integer"}

batch_analysis = True  #@param {type:"boolean"}
batch_folder = '/content/gdrive/MyDrive/data/otto_12'  #@param {type:"string"}

%cd /content/LibraryAnalysis_gui/scripts/
# run the analysis
arguments = ['-wt', wt_name, '-minq', str(minimum_read_quality), '-minb', str(minimum_base_quality)]
if run_variant:
  arguments.append('-v')
if indel_analysis:
  arguments.append('-i')
if full_length_analysis:
  arguments.append('-vfull')
if long_read_analysis:
  arguments.append('-pb')
if run_correlation:
  arguments.append('-c')
if parallel_threads > 1:
  arguments.append('-par')
  arguments.append(str(parallel_threads))
if domains:
  arguments.append('-d')
  arguments.append(domains)
if designed_mutations:
  arguments.append('-m')
  arguments.append(designed_mutations)

if batch_analysis:
  import re
  import os
  paired_files = {}
  for file in os.listdir(batch_folder):
    if '.fastq' in file and 'corrected' not in file:
      try:
        paired_files[re.sub(r'_R\d_', '', file)].append(os.path.join(batch_folder, file))
      except KeyError:
        paired_files[re.sub(r'_R\d_', '', file)] = [os.path.join(batch_folder, file)]
  for key, value in paired_files.items():
    assert len(value) < 3, 'Too many files'
    tmp_args = arguments.copy()
    tmp_args.append('-s')
    tmp_args.append(value[0])
    tmp_args.append('-p')
    tmp_args.append(value[1])
    command = ' '.join(tmp_args)
    #print(command)
    !python LibAn.py {command}
else:
  arguments.append('-s')
  arguments.append(read1_name)
  arguments.append('-p')
  arguments.append(read2_name)
  command = ' '.join(arguments)
  !python LibAn.py {command}



In [ ]:
#@title 6. Download the results
# add files to output_dir
output_dir = '/content'
!zip -q -r {output_dir}.zip {output_dir} -x {output_dir}/LibraryAnalysis/\* {output_dir}/sample_data/\*
files.download(f'{output_dir}.zip')